In [11]:
import warnings
warnings.filterwarnings("ignore")
import os
import dotenv

import json
import pandas as pd
import requests

dotenv.load_dotenv()

import redis
from langchain.vectorstores.redis import Redis
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
redis_host = os.getenv("REDIS_HOST")
redis_password = os.getenv("REDIS_PASS")


r = redis.Redis(
  host=redis_host,
  port=14266,
  password=redis_password,
 )

## Preparing Data

In [146]:

def get_data():
	url = "https://eeg-backend-hfehdmd4hxfagsgu.canadacentral-01.azurewebsites.net/api/users/product"

	response = requests.get(url)

	return response.json()

In [147]:
data = get_data()
print(data)

[{'chemicalProperties': {}, 'seller': {'name': 'ChemWorld Ltd.', 'contactInfo': '', 'phone': '', 'address': '', 'user': 1}, 'productType': 'Industrial Supply', 'productId': 'CHM-56789-BBB', 'productName': 'Sodium Hydroxide (Caustic Soda)', 'productIdentification': '99% Pure NaOH', 'purityComposition': '99% NaOH, 1% Moisture', 'quantity': 1200, 'unit': 'kg', 'price': '20.00', 'currency': 'USD', 'rating': '4.70', 'category': 'Industrial Supplies', 'subCategory': 'Chemicals', 'minimumOrderQuantity': 10, 'availabilityStatus': 'In Stock', 'location': 'Detroit, USA', 'deliveryTime': '2-4 business days', 'images': ['https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi5.walmartimages.com%2Fasr%2Ff7151025-f254-4c23-ac71-3b5ee8a4486f_1.06599ced82e3e88315e88b4048430fac.jpeg&f=1&nofb=1&ipt=be0e8019ee80faaaa7dac31cdeeb924453906bf2e2711a5e0a1afdcb7679d521&ipo=images'], 'manufactureDate': '2024-02-01', 'expiryDate': '2025-02-01', 'shippingCost': '25.00'}, {'physicalProperties': {'length': '1

In [148]:
#create a dataframe with the keys as columns and values as rows
df = pd.DataFrame(data)
df.head()

,chemicalProperties,seller,productType,productId,productName,productIdentification,purityComposition,quantity,unit,price,...,minimumOrderQuantity,availabilityStatus,location,deliveryTime,images,manufactureDate,expiryDate,shippingCost,physicalProperties,discount
0,{},"{'name': 'ChemWorld Ltd.', 'contactInfo': '', ...",Industrial Supply,CHM-56789-BBB,Sodium Hydroxide (Caustic Soda),99% Pure NaOH,"99% NaOH, 1% Moisture",1200,kg,20.00,...,10,In Stock,"Detroit, USA",2-4 business days,[https://external-content.duckduckgo.com/iu/?u...,2024-02-01,2025-02-01,25.00,NaN,NaN
1,{'carbonContent': '0.25%'},"{'name': 'SteelMakers Inc.', 'contactInfo': ''...",Construction Material,TMT-22334-CCC,TMT Iron Rod,Grade Fe 500D,"99% Iron, 1% Alloy",2000,rods,50.00,...,50,In Stock,"Pittsburgh, USA",5-7 business days,[https://waritigiafrica.com/wp-content/uploads...,2023-11-05,NaN,100.00,"{'length': '12 meters', 'diameter': '16 mm', '...",1.0
2,{},"{'name': 'WeldPro Equipment', 'contactInfo': '...",Industrial Supply,WLD-77889-DDD,MIG Welding Machine,Model MIG-350,NaN,100,pieces,500.00,...,1,In Stock,"Houston, USA",3-5 business days,[https://m.media-amazon.com/images/I/71uvCWPWM...,2024-01-01,NaN,50.00,{'weight': '15 kg'},3.0
3,{},"{'name': 'PipeMasters LLC', 'contactInfo': '',...",Construction Material,PVC-33445-EEE,PVC Water Supply Pipe,"NSF Certified, Class 315",100% Virgin PVC,3000,pipes,12.00,...,20,In Stock,"Miami, USA",5-7 business days,[https://kennedys.net.au/wp-content/uploads/20...,2024-05-01,NaN,20.00,"{'length': '6 meters', 'diameter': '50 mm', 'w...",4.0
4,{},"{'name': 'PaintPro Co.', 'contactInfo': '', 'p...",Industrial Supply,PAINT-88900-FFF,Industrial Epoxy Paint,"Corrosion-Resistant, ISO 12944 Compliant","95% Epoxy Resin, 5% Solvent",500,buckets,300.00,...,1,In Stock,"Detroit, USA",3-5 business days,[https://5.imimg.com/data5/SELLER/Default/2023...,2024-07-01,2025-07-01,35.00,{'weight': '25 kg'},5.0


In [149]:
df.fillna("Unknown", inplace=True)
df.head()

,chemicalProperties,seller,productType,productId,productName,productIdentification,purityComposition,quantity,unit,price,...,minimumOrderQuantity,availabilityStatus,location,deliveryTime,images,manufactureDate,expiryDate,shippingCost,physicalProperties,discount
0,{},"{'name': 'ChemWorld Ltd.', 'contactInfo': '', ...",Industrial Supply,CHM-56789-BBB,Sodium Hydroxide (Caustic Soda),99% Pure NaOH,"99% NaOH, 1% Moisture",1200,kg,20.00,...,10,In Stock,"Detroit, USA",2-4 business days,[https://external-content.duckduckgo.com/iu/?u...,2024-02-01,2025-02-01,25.00,Unknown,Unknown
1,{'carbonContent': '0.25%'},"{'name': 'SteelMakers Inc.', 'contactInfo': ''...",Construction Material,TMT-22334-CCC,TMT Iron Rod,Grade Fe 500D,"99% Iron, 1% Alloy",2000,rods,50.00,...,50,In Stock,"Pittsburgh, USA",5-7 business days,[https://waritigiafrica.com/wp-content/uploads...,2023-11-05,Unknown,100.00,"{'length': '12 meters', 'diameter': '16 mm', '...",1.0
2,{},"{'name': 'WeldPro Equipment', 'contactInfo': '...",Industrial Supply,WLD-77889-DDD,MIG Welding Machine,Model MIG-350,Unknown,100,pieces,500.00,...,1,In Stock,"Houston, USA",3-5 business days,[https://m.media-amazon.com/images/I/71uvCWPWM...,2024-01-01,Unknown,50.00,{'weight': '15 kg'},3.0
3,{},"{'name': 'PipeMasters LLC', 'contactInfo': '',...",Construction Material,PVC-33445-EEE,PVC Water Supply Pipe,"NSF Certified, Class 315",100% Virgin PVC,3000,pipes,12.00,...,20,In Stock,"Miami, USA",5-7 business days,[https://kennedys.net.au/wp-content/uploads/20...,2024-05-01,Unknown,20.00,"{'length': '6 meters', 'diameter': '50 mm', 'w...",4.0
4,{},"{'name': 'PaintPro Co.', 'contactInfo': '', 'p...",Industrial Supply,PAINT-88900-FFF,Industrial Epoxy Paint,"Corrosion-Resistant, ISO 12944 Compliant","95% Epoxy Resin, 5% Solvent",500,buckets,300.00,...,1,In Stock,"Detroit, USA",3-5 business days,[https://5.imimg.com/data5/SELLER/Default/2023...,2024-07-01,2025-07-01,35.00,{'weight': '25 kg'},5.0


In [150]:
df["chemicalProperties"] = df["chemicalProperties"].apply(lambda x: "Unknown" if len(x) == 0 else x)
df.head()

,chemicalProperties,seller,productType,productId,productName,productIdentification,purityComposition,quantity,unit,price,...,minimumOrderQuantity,availabilityStatus,location,deliveryTime,images,manufactureDate,expiryDate,shippingCost,physicalProperties,discount
0,Unknown,"{'name': 'ChemWorld Ltd.', 'contactInfo': '', ...",Industrial Supply,CHM-56789-BBB,Sodium Hydroxide (Caustic Soda),99% Pure NaOH,"99% NaOH, 1% Moisture",1200,kg,20.00,...,10,In Stock,"Detroit, USA",2-4 business days,[https://external-content.duckduckgo.com/iu/?u...,2024-02-01,2025-02-01,25.00,Unknown,Unknown
1,{'carbonContent': '0.25%'},"{'name': 'SteelMakers Inc.', 'contactInfo': ''...",Construction Material,TMT-22334-CCC,TMT Iron Rod,Grade Fe 500D,"99% Iron, 1% Alloy",2000,rods,50.00,...,50,In Stock,"Pittsburgh, USA",5-7 business days,[https://waritigiafrica.com/wp-content/uploads...,2023-11-05,Unknown,100.00,"{'length': '12 meters', 'diameter': '16 mm', '...",1.0
2,Unknown,"{'name': 'WeldPro Equipment', 'contactInfo': '...",Industrial Supply,WLD-77889-DDD,MIG Welding Machine,Model MIG-350,Unknown,100,pieces,500.00,...,1,In Stock,"Houston, USA",3-5 business days,[https://m.media-amazon.com/images/I/71uvCWPWM...,2024-01-01,Unknown,50.00,{'weight': '15 kg'},3.0
3,Unknown,"{'name': 'PipeMasters LLC', 'contactInfo': '',...",Construction Material,PVC-33445-EEE,PVC Water Supply Pipe,"NSF Certified, Class 315",100% Virgin PVC,3000,pipes,12.00,...,20,In Stock,"Miami, USA",5-7 business days,[https://kennedys.net.au/wp-content/uploads/20...,2024-05-01,Unknown,20.00,"{'length': '6 meters', 'diameter': '50 mm', 'w...",4.0
4,Unknown,"{'name': 'PaintPro Co.', 'contactInfo': '', 'p...",Industrial Supply,PAINT-88900-FFF,Industrial Epoxy Paint,"Corrosion-Resistant, ISO 12944 Compliant","95% Epoxy Resin, 5% Solvent",500,buckets,300.00,...,1,In Stock,"Detroit, USA",3-5 business days,[https://5.imimg.com/data5/SELLER/Default/2023...,2024-07-01,2025-07-01,35.00,{'weight': '25 kg'},5.0


In [151]:
columns = df.columns
corpus = []
for i in range(df.shape[0]):
        text = ""
        for col in columns:
                text += col+ ": " + str(df[col][i]) + " "
        corpus.append(text)
corpus[:2]

["chemicalProperties: Unknown seller: {'name': 'ChemWorld Ltd.', 'contactInfo': '', 'phone': '', 'address': '', 'user': 1} productType: Industrial Supply productId: CHM-56789-BBB productName: Sodium Hydroxide (Caustic Soda) productIdentification: 99% Pure NaOH purityComposition: 99% NaOH, 1% Moisture quantity: 1200 unit: kg price: 20.00 currency: USD rating: 4.70 category: Industrial Supplies subCategory: Chemicals minimumOrderQuantity: 10 availabilityStatus: In Stock location: Detroit, USA deliveryTime: 2-4 business days images: ['https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi5.walmartimages.com%2Fasr%2Ff7151025-f254-4c23-ac71-3b5ee8a4486f_1.06599ced82e3e88315e88b4048430fac.jpeg&f=1&nofb=1&ipt=be0e8019ee80faaaa7dac31cdeeb924453906bf2e2711a5e0a1afdcb7679d521&ipo=images'] manufactureDate: 2024-02-01 expiryDate: 2025-02-01 shippingCost: 25.00 physicalProperties: Unknown discount: Unknown ",
 "chemicalProperties: {'carbonContent': '0.25%'} seller: {'name': 'SteelMakers Inc.

## Product Summaries

In [152]:
def create_prod_summary(data):
	llm = ChatOpenAI(
		model="gpt-3.5-turbo",
		temperature=0,
		max_tokens=None,
		timeout=None,
		max_retries=2,
		api_key=OPENAI_API_KEY,
	)

	message = f"Here is a product data {data}. Your job is to create a listing of the entire product. Mention all the features and details present in the data."

	response = llm.invoke(message)
	return response.content

In [153]:
summary = []
for prod in corpus:
	summary.append(create_prod_summary(prod))

summary[:2]

['Product Listing:\n- Product ID: CHM-56789-BBB\n- Product Name: Sodium Hydroxide (Caustic Soda)\n- Seller: ChemWorld Ltd.\n- Contact Information: Not provided\n- Phone: Not provided\n- Address: Not provided\n- User: 1\n- Product Type: Industrial Supply\n- Purity: 99% Pure NaOH\n- Composition: 99% NaOH, 1% Moisture\n- Quantity: 1200 kg\n- Price: $20.00 USD\n- Rating: 4.70\n- Category: Industrial Supplies\n- Subcategory: Chemicals\n- Minimum Order Quantity: 10\n- Availability Status: In Stock\n- Location: Detroit, USA\n- Delivery Time: 2-4 business days\n- Images: [https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi5.walmartimages.com%2Fasr%2Ff7151025-f254-4c23-ac71-3b5ee8a4486f_1.06599ced82e3e88315e88b4048430fac.jpeg&f=1&nofb=1&ipt=be0e8019ee80faaaa7dac31cdeeb924453906bf2e2711a5e0a1afdcb7679d521&ipo=images]\n- Manufacture Date: 2024-02-01\n- Expiry Date: 2025-02-01\n- Shipping Cost: $25.00\n- Physical Properties: Unknown\n- Discount: Unknown',
 'Product Listing:\n- Product ID

## Vector Embeddings

In [4]:
embed_model = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

### Redis DB

In [6]:
redis_url = f"redis://:{redis_password}@{redis_host}:14266"
index_name = "product_index"

In [7]:
schema = {
    "summary": "TEXT",  # Field for product summaries (TEXT type)
    "id": "NUMERIC",  # Metadata (NUMERIC type for IDs)
    "embedding": {  # Field for embeddings (VECTOR type)
        "TYPE": "FLOAT32",  # Data type of vector
        "DIM": 768,  # Dimension of the vectors (adjust according to your model)
        "DISTANCE_METRIC": "COSINE"  # Similarity metric for search
    }
}

In [237]:
# metadata = [{"id": i} for i in range(len(summary))]
# redis_vectorstore = Redis.from_texts(
#   texts=summary,
#   metadatas=metadata,
#   embedding=embed_model,
#   index_name=index_name,
#   redis_url=redis_url,
# )

In [8]:
redis_instance = Redis.from_existing_index(
  embedding=embed_model,
  index_name=index_name,
  redis_url=redis_url,
  schema=schema,
)

In [248]:
# Redis.drop_index(index_name, redis_url=redis_url, delete_documents=True)

## LLM Query

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_redis import RedisChatMessageHistory

In [9]:
question_template = """You are an expert in summarizing questions. 
					Your goal is to reduce a question to its simplest form while retaining the semantic meaning. 
             		Try to be as deterministic as possible
					Below is the question:
					{question}
					Output will be a semantically similar question that will be used to query an existing database."""

prompt = """You are an expert in answering questions on product related queries.
					You have to provide the most relevant answer to the question based on the product data.
					If multiple products are relevant, you can provide a list of those products. 
					Here is the product data:
					{data}
					And here is the question:
					{question}
					For queries like Hi or Hello you can respond with greetings or a polite message."""



def llm_response(question):
	llm = ChatOpenAI(
		model="gpt-3.5-turbo",
		temperature=0,
		max_tokens=None,
		timeout=None,
		max_retries=2,
		api_key=OPENAI_API_KEY,
	)
	
	question = question_template.format(question=question)
	modified_question = llm.invoke(question).content
	tokenized_query = modified_question.split(" ")
	redis_result = redis_instance.similarity_search(query=modified_question, k=5)
	final_result = ""
	for res in redis_result:
		final_result += res.page_content + "\n"
	answer_msg = prompt.format(data=final_result, question=modified_question)
	response = llm.invoke(answer_msg)
	return response.content

In [12]:
x = llm_response("List the iron items available")
print(x)

The iron items that are currently in stock are:
1. Iron Ore - Bulk by OreMining Corp.
2. TMT Iron Rod by SteelMakers Inc.


In [21]:
prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert in answering questions about products. 
                     Answer based on the retrieved product data below:
                     {context}
                     
                     For greetings like "Hi" or "Hello", respond politely.
                     If multiple products are relevant, list all of them.
                     If you're not sure about something, say so."""),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}")
    ])

In [29]:
# First define the retrieval function
def retrieve_docs(question):
    modified_question = llm.invoke(question_template.format(question=question)).content
    redis_result = redis_instance.similarity_search(query=modified_question, k=5)
    return "\n".join(res.page_content for res in redis_result)

llm = ChatOpenAI(
		model="gpt-3.5-turbo",
		temperature=0,
		max_tokens=None,
		timeout=None,
		max_retries=2,
		api_key=OPENAI_API_KEY,
	)

chain = prompt | llm

# Create Redis history manager
def get_redis_history(session_id: str) -> BaseChatMessageHistory:
    return RedisChatMessageHistory(
        session_id=session_id,
		redis_client=r,
    )

# Create chain with history
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_redis_history,
    input_messages_key="question",
    history_messages_key="history"
)

# Example usage:
def chat_with_memory(question: str, session_id: str):
    # Get context from Redis vector store
    context = retrieve_docs(question)
    
    # Run chain with memory
    response = chain_with_history.invoke(
        {"question": question, "context": context},
        config={"configurable": {"session_id": session_id}}
    )
    return response.content

# Test the chat
session_id = "test_session_1"
print(chat_with_memory("Hi my name is Ronit", session_id))

Hello Ronit! How can I assist you today?


In [30]:
print(chat_with_memory("What is my name?", session_id)) 

Your name is Ronit. How can I assist you further, Ronit?


In [40]:
import requests

url = "http://localhost:8000/api/chat"
data = {
    "question": "What is the price of the iron items?",
    "session_id": "test_session_1"
}

response = requests.post(url, json=data)

if response.status_code == 200:
    result = response.json()
    answer = result["answer"]
    print("Bot:", answer)
else:
    print("Error:", response.status_code, response.text)

Bot: The prices of the iron items available are as follows:

1. **Iron Ore - Bulk**
   - Price: $150.00 per ton

2. **TMT Iron Rod**
   - Price: $50.00 per rod

If you have any more questions or need further details, feel free to ask.
